In [54]:
import pandas as pd
import sqlalchemy as sa
import numpy as np

import bokeh.io as bio
from bokeh.plotting import figure as bfig
from bokeh.models import WMTSTileSource, ColumnDataSource, LinearColorMapper, HoverTool
from bokeh.tile_providers import CARTODBPOSITRON

# SAT 2012 data

In [45]:
###### set up SAT 2012 dataframe from databases
engine = sa.create_engine('sqlite:///db/nycedudata.db')
meta = sa.MetaData()

SAT_2012_query = "SELECT Schools.dbn, Schools.school_name, Schools.latitude, Schools.longitude, SAT2012.total_rm \
                  FROM Schools INNER JOIN (SELECT * FROM SAT WHERE school_year = 2012) AS SAT2012 \
                  ON SAT2012.dbn = Schools.dbn;"

sat2012_df = pd.read_sql_query(SAT_2012_query, engine)

k = 6378137
sat2012_df['latitude'] = sat2012_df['latitude'].apply( lambda x: np.log(np.tan((90+float(x)) * np.pi/360.0))*k )
sat2012_df['longitude'] = sat2012_df['longitude'].apply( lambda x: float(x) * (k * np.pi/180) )
sat2012_df['total_rm'] = sat2012_df['total_rm'].apply( lambda x: int(x) )

# sat2012_df.head()


,dbn,school_name,latitude,longitude,total_rm
0,01m292,orchard collegiate academy,4.970201e+06,-8.236121e+06,759
1,01m448,university neighborhood high school,4.970013e+06,-8.235917e+06,806
2,01m450,east side community school,4.972538e+06,-8.235700e+06,779
3,01m458,forsyth satellite academy,4.971499e+06,-8.236610e+06,815
4,01m509,marta valle high school,4.971214e+06,-8.236044e+06,823


In [46]:
padding = 10

south = np.min(sat2012_df['latitude']) - padding
print(south)
north = np.max(sat2012_df['latitude']) + padding
print(north)
west = np.min(sat2012_df['longitude']) - padding
print(west)
east = np.max(sat2012_df['longitude']) + padding
print(east)

4942972.56041
4995895.46943
-8259029.22316
-8207301.65364


In [60]:
p = bfig(tools='pan, wheel_zoom', x_range=(west,east), y_range=(south,north), 
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(CARTODBPOSITRON)

data_source = ColumnDataSource(sat2012_df)
color_map = LinearColorMapper(palette='Magma256', low=600, high=1200))

p.circle(x='longitude', y='latitude', source=data_source,fill_color={'field':'total_rm', 'transform':color_map}, size=10)
p.add_tools(HoverTool(tooltips=[('DBN','@dbn'),('Name','@school_name'),('Score','@total_rm')]))
bio.show(p)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: score, x, y [renderer: GlyphRenderer(id='03e2cfc3-9a43-46f6-8b71-6abad425c9ac', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: x, y [renderer: GlyphRenderer(id='87c81c01-8dc3-4df7-b591-a85df4924408', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: x, y [renderer: GlyphRenderer(id='c07bb362-d333-4f4e-9ffa-2cb0aa46907a', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: score, x, y [renderer: GlyphRenderer(id='e6279e28-dc32-4f8e-827c-9848b725bbb9', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: total_rm, x, y [renderer: GlyphRenderer(id='71e52f30-216f-444e-889b-d4b12a8d74d4', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to 

# Regents Data